In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

## 1. Data Generation (simulating data for biodiversity loss)


In [2]:
def generate_biodiversity_data(n_samples=1000):
    np.random.seed(42)
    
    # Features: habitat_loss, pollution, climate_change, invasive_species, overexploitation
    habitat_loss = np.random.uniform(0, 1, n_samples)
    pollution = np.random.uniform(0, 1, n_samples)
    climate_change = np.random.uniform(0, 1, n_samples)
    invasive_species = np.random.uniform(0, 1, n_samples)
    overexploitation = np.random.uniform(0, 1, n_samples)
    
    # Target: biodiversity_index (simplified model)
    biodiversity_index = (
        1 - (0.3 * habitat_loss +
             0.2 * pollution +
             0.2 * climate_change +
             0.15 * invasive_species +
             0.15 * overexploitation +
             np.random.normal(0, 0.05, n_samples))
    )
    biodiversity_index = np.clip(biodiversity_index, 0, 1)
    
    data = pd.DataFrame({
        'habitat_loss': habitat_loss,
        'pollution': pollution,
        'climate_change': climate_change,
        'invasive_species': invasive_species,
        'overexploitation': overexploitation,
        'biodiversity_index': biodiversity_index
    })
    
    return data

## 2. Data Preprocessing


In [3]:
def preprocess_data(data):
    X = data[['habitat_loss', 'pollution', 'climate_change', 'invasive_species', 'overexploitation']]
    y = data['biodiversity_index']
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test, scaler

## 3. Model Creation


In [4]:
def create_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=input_shape),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

## 4. Model Training


In [5]:
def train_model(model, X_train, y_train):
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)
    return history

## 5. Question Generation


In [6]:
def generate_question(model, scaler, question_type='multiple_choice'):
    # Generate random input
    input_data = np.random.rand(1, 5)
    scaled_input = scaler.transform(input_data)
    
    # Predict biodiversity index
    predicted_index = model.predict(scaled_input)[0][0]
    
    # Create question based on the prediction
    if question_type == 'multiple_choice':
        question = "Given the following environmental conditions:\n"
        question += f"Habitat Loss: {input_data[0][0]:.2f}\n"
        question += f"Pollution Level: {input_data[0][1]:.2f}\n"
        question += f"Climate Change Impact: {input_data[0][2]:.2f}\n"
        question += f"Invasive Species Presence: {input_data[0][3]:.2f}\n"
        question += f"Overexploitation: {input_data[0][4]:.2f}\n\n"
        question += "What is the most likely impact on biodiversity?\n"
        
        options = [
            f"A) High biodiversity (index >{predicted_index+0.2:.2f})",
            f"B) Moderate biodiversity (index {predicted_index-0.1:.2f} - {predicted_index+0.1:.2f})",
            f"C) Low biodiversity (index <{predicted_index-0.2:.2f})",
            "D) No significant impact on biodiversity"
        ]
        
        correct_answer = 'B'
        
        return question, options, correct_answer
    
    elif question_type == 'true_false':
        statement = "Given the environmental conditions, "
        if predicted_index > 0.5:
            statement += "the biodiversity index is likely to be high."
            correct_answer = True
        else:
            statement += "the biodiversity index is likely to be low."
            correct_answer = False
        
        return statement, correct_answer

## 6. Main Execution


In [7]:
if __name__ == "__main__":
    # Generate and preprocess data
    data = generate_biodiversity_data()
    X_train, X_test, y_train, y_test, scaler = preprocess_data(data)
    
    # Create and train model
    model = create_model((X_train.shape[1],))
    history = train_model(model, X_train, y_train)
    
    # Evaluate model
    test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test MAE: {test_mae:.4f}")
    
    # Generate questions
    print("\nMultiple Choice Question:")
    mc_question, mc_options, mc_answer = generate_question(model, scaler, 'multiple_choice')
    print(mc_question)
    for option in mc_options:
        print(option)
    print(f"Correct Answer: {mc_answer}")
    
    print("\nTrue/False Question:")
    tf_statement, tf_answer = generate_question(model, scaler, 'true_false')
    print(tf_statement)
    print(f"Correct Answer: {tf_answer}")

/Users/rajesh/.pyenv/versions/3.11.4/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test MAE: 0.0410

Multiple Choice Question:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Given the following environmental conditions:
Habitat Loss: 0.07
Pollution Level: 0.55
Climate Change Impact: 0.59
Invasive Species Presence: 0.65
Overexploitation: 0.26

What is the most likely impact on biodiversity?

A) High biodiversity (index >0.82)
B) Moderate biodiversity (index 0.52 - 0.72)
C) Low biodiversity (index <0.42)
D) No significant impact on biodiversity
Correct Answer: B

True/False Question:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Given the environmental conditions, the biodiversity index is likely to be low.
Correct Answer: False


/Users/rajesh/.pyenv/versions/3.11.4/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/rajesh/.pyenv/versions/3.11.4/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


## 7. Interactive Quiz Function


In [8]:
def take_quiz(model, scaler, num_questions=5):
    score = 0
    for i in range(num_questions):
        print(f"\nQuestion {i+1}:")
        if np.random.rand() > 0.5:
            # Multiple Choice Question
            question, options, correct_answer = generate_question(model, scaler, 'multiple_choice')
            print(question)
            for option in options:
                print(option)
            user_answer = input("Your answer (A/B/C/D): ").upper()
            if user_answer == correct_answer:
                print("Correct!")
                score += 1
            else:
                print(f"Incorrect. The correct answer is {correct_answer}.")
        else:
            # True/False Question
            statement, correct_answer = generate_question(model, scaler, 'true_false')
            print(statement)
            user_answer = input("True or False? (T/F): ").upper() == 'T'
            if user_answer == correct_answer:
                print("Correct!")
                score += 1
            else:
                print(f"Incorrect. The correct answer is {correct_answer}.")
    
    print(f"\nQuiz completed! Your score: {score}/{num_questions}")

## 8. Run the quiz


In [9]:
if __name__ == "__main__":
    print("\nWelcome to the Biodiversity Loss Impact Quiz!")
    take_quiz(model, scaler)


Welcome to the Biodiversity Loss Impact Quiz!

Question 1:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


/Users/rajesh/.pyenv/versions/3.11.4/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Given the following environmental conditions:
Habitat Loss: 0.71
Pollution Level: 0.21
Climate Change Impact: 0.57
Invasive Species Presence: 0.87
Overexploitation: 0.43

What is the most likely impact on biodiversity?

A) High biodiversity (index >0.64)
B) Moderate biodiversity (index 0.34 - 0.54)
C) Low biodiversity (index <0.24)
D) No significant impact on biodiversity
Incorrect. The correct answer is B.

Question 2:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


/Users/rajesh/.pyenv/versions/3.11.4/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Given the following environmental conditions:
Habitat Loss: 0.45
Pollution Level: 0.14
Climate Change Impact: 0.39
Invasive Species Presence: 0.95
Overexploitation: 0.32

What is the most likely impact on biodiversity?

A) High biodiversity (index >0.77)
B) Moderate biodiversity (index 0.47 - 0.67)
C) Low biodiversity (index <0.37)
D) No significant impact on biodiversity
Incorrect. The correct answer is B.

Question 3:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


...

Given the following environmental conditions:
Habitat Loss: 0.88
Pollution Level: 0.35
Climate Change Impact: 0.08
Invasive Species Presence: 0.72
Overexploitation: 0.31

What is the most likely impact on biodiversity?

A) High biodiversity (index >0.69)
B) Moderate biodiversity (index 0.39 - 0.59)
C) Low biodiversity (index <0.29)
D) No significant impact on biodiversity
Incorrect. The correct answer is B.

Question 4:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


/Users/rajesh/.pyenv/versions/3.11.4/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Given the following environmental conditions:
Habitat Loss: 0.44
Pollution Level: 0.49
Climate Change Impact: 0.78
Invasive Species Presence: 0.38
Overexploitation: 0.44

What is the most likely impact on biodiversity?

A) High biodiversity (index >0.69)
B) Moderate biodiversity (index 0.39 - 0.59)
C) Low biodiversity (index <0.29)
D) No significant impact on biodiversity
Incorrect. The correct answer is B.

Question 5:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


/Users/rajesh/.pyenv/versions/3.11.4/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Given the environmental conditions, the biodiversity index is likely to be low.
Correct!

Quiz completed! Your score: 1/5


## 9. Educational Information


In [10]:
def display_biodiversity_info():
    print("\nBiodiversity Loss: Key Points")
    print("1. Definition: Biodiversity loss refers to the reduction of variety in living species within an ecosystem.")
    print("2. Main Causes:")
    print("   - Habitat Loss: Destruction of natural environments")
    print("   - Pollution: Contamination of air, water, and soil")
    print("   - Climate Change: Altering temperature and weather patterns")
    print("   - Invasive Species: Non-native species outcompeting local ones")
    print("   - Overexploitation: Excessive hunting, fishing, or harvesting")
    print("3. Impacts:")
    print("   - Ecosystem Instability: Reduced resilience to environmental changes")
    print("   - Loss of Ecosystem Services: Affecting water purification, pollination, etc.")
    print("   - Economic Losses: Impacting industries like agriculture and tourism")
    print("   - Health Risks: Potential increase in zoonotic diseases")
    print("4. Conservation Efforts:")
    print("   - Protected Areas: Establishing and maintaining nature reserves")
    print("   - Sustainable Practices: Promoting responsible resource use")
    print("   - Restoration Projects: Rehabilitating damaged ecosystems")
    print("   - Education: Raising awareness about biodiversity importance")
    print("\nUnderstanding these factors is crucial for addressing biodiversity loss effectively.")


## 10. Extended Main Execution


In [11]:
if __name__ == "__main__":
    print("\nWelcome to the Biodiversity Loss Educational Program!")
    
    while True:
        print("\nChoose an option:")
        print("1. Take a quiz")
        print("2. Learn about biodiversity loss")
        print("3. Exit")
        
        choice = input("Enter your choice (1/2/3): ")
        
        if choice == '1':
            take_quiz(model, scaler)
        elif choice == '2':
            display_biodiversity_info()
        elif choice == '3':
            print("Thank You. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")


Welcome to the Biodiversity Loss Educational Program!

Choose an option:
1. Take a quiz
2. Learn about biodiversity loss
3. Exit
Invalid choice. Please try again.

Choose an option:
1. Take a quiz
2. Learn about biodiversity loss
3. Exit
Invalid choice. Please try again.

Choose an option:
1. Take a quiz
2. Learn about biodiversity loss
3. Exit
Invalid choice. Please try again.

Choose an option:
1. Take a quiz
2. Learn about biodiversity loss
3. Exit
Thank you for learning about biodiversity loss. Goodbye!


In [2]:
import joblib
from sklearn.ensemble import RandomForestClassifier

# Create a scikit-learn model
model = RandomForestClassifier()

# Train the model on your data

# Save the model to a file
joblib.dump(model, 'BL.pkl')

['BL.pkl']